# Prepare Stack NLP from txt to ner app

Neste script iremos transformar txt selecionado num arquivo em formato csv para a marcação de entidades nomeadas.

Usaremos o pacote padrão do python para organizar txt em uma string

Usaremos o pacote NLTK para a detecção de sentenças e para tokenizar

Para o treinamento e uso do Postagger, usaremos o PerceptronTagger do NLTK para treinar o modelo usando o corpus Floresta

In [ ]:
import re
import nltk
from nltk import tokenize
from nltk.tag.perceptron import PerceptronTagger
nltk.download('floresta')
from nltk.corpus import floresta

Aqui iremos abrir o arquivo txt em uma variavel chamada data


onde tem '../TXT/jardim.txt' você pode mudar o 'jardim' para o nome do txt que você criou no passo 1

In [ ]:
with open('../TXT/jardim.txt', 'r') as myfile:
    data = myfile.read()

Ela já está em forma de string, porém cheia de ruído, você pode visualizar ela executando a célula abaixo

In [ ]:
data

Está cheio de \n e palavras quebradas! iremos corrigir isso usando regex (re) o pacote de expressões regulares do python

lembrando que números de páginas e outros ruídos devem ser removidos manualmente!



primeiro iremos usar a função sub para remover as linhas em branco

In [ ]:
data = re.sub('\n\n','\n',data)

depois usaremos a mesma função para juntar as palavras que foram hifenizadas por quebra de texto.

ex:

pe-

dro

dentro da string ela está representada por 'pe-\ndro', se removermos o '-\n' teremos 'pedro' como resultado

In [ ]:
data = re.sub('-\n','',data)

finalmente podemos remover as quebras de linhas

In [ ]:
data = re.sub('\n','',data)

Pronto! nossa string está limpa! você podem visualizar chamando a variavel data

In [ ]:
data

## Sentece Detector

Agora iremos separar nossa string em sentenças

a célula abaixo carrega um modelo para o português

In [ ]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')

Agora iremos aplicar o modelo a nossa variavel data:

In [ ]:
sentences = sent_tokenizer.tokenize(data)

você pode visualizar as sentenças chamando a variavel:

In [ ]:
sentences[0:3]

## Tokenizer

agora iremos separar cada sentença em tokens


para isso criaremos um vetor vazio e salvaremos nele o output da função tokenize.word_tokenize para cada sentença

a função tokenize.word_tokenize recebe um sentença e retorna um vetor de tokens

In [ ]:
tokens = []
for sent in sentences:
    tokens.append(tokenize.word_tokenize(sent, language='portuguese'))  

você pode visualizar os tokens chamando a variavel:

In [ ]:
tokens[0][0:10]

## Postag

Uma parte do codigo abaixo foi copiado do site: http://www.nltk.org/howto/portuguese_en.html referente a preparação do corpus floresta. Nele carregamos o corpus, organizamos os tokens e separamos em treino e teste, essa separação é fundamental, pois como vamos treinar nosso próprio modelo de Postag será avaliado os resultados.

In [ ]:
def simplify_tag(t):
    if "+" in t:
        return t[t.index("+")+1:]
    else:
        return t


tsents = floresta.tagged_sents()
tsents = [[(w.lower(),simplify_tag(t)) for (w,t) in sent] for sent in tsents if sent]
train = tsents[100:]
test = tsents[:100]

Iremos agora carregar a função que irá fazer o treinamento

Ela é baseada em Perceptrons, um conceito fundamental de inteligência computacional que tenta simular a operação de um neurônio sendo a base das redes neurais modernas.

Referências https://en.wikipedia.org/wiki/Perceptron 

In [ ]:
tagger = PerceptronTagger(load=False)

Agora iremos treinar o modelo (...)

In [ ]:
tagger.train(train)

Vamos avaliar com a função evaluate. O score varia de 0 a 1, quanto mais proximo de 1 melhor!

In [ ]:
tagger.evaluate(test)

Agora iremos usar o nosso modelo para fazer o PoStag em nossas sentenças

iremos salvar em outro vetor chamado sentences_tagged pra não misturar as coisas rs

In [ ]:
sentences_tagged = []
for i in tokens:
    sentences_tagged.append(tagger.tag(i))

você pode visualizar o PoStag chamando a variavel:

In [ ]:
sentences_tagged[0][0:10]

## Organizar e Salvar

agora iremos organizar tudo em uma matriz em que a primera coluna é o numero da sentença, a segunda o token, a terceira o PoStag e a quarta a NERtag. Como não taggeamos ainda colocaremos todas igual a 0

In [ ]:
matriz = []
for i in range(0,len(sentences_tagged)):
    for j in range(0,len(sentences_tagged[i])):
        matriz.append([i,sentences_tagged[i][j][0],sentences_tagged[i][j][1],0])

você pode visualizar a matriz chamando a variavel:

In [ ]:
matriz[0:10]

Agora iremos transformar essa matriz em um dataframe usando o pacote Pandas

dataframe é uma estrutura melhor para visualizar e guardar diferentes tipos de dados

inclusive podemos dizer o que cada coluna é dentro da estrutura

In [ ]:
import pandas as pd
df = pd.DataFrame(matriz,columns=['Sentence','Token','PosTag','NERtag'])

você pode visualizar o resultado usando a função head()

In [ ]:
df.head()

Agora iremos salvar esse dataframe em um csv!

você pode mudar a parte 'jardimNER' para o nome do seu livro

In [ ]:
df.to_csv('../NER/jardimNER.csv',index=False)

## Valeu!

### made by Vinicius Sampaio/ Mardônio França from Boitatá